In [1]:
using LinearAlgebra,CSV, DataFrames

In [6]:
retail = "./Data/generated_data_complete.csv" #468×13 DataFrame
retail = DataFrame(CSV.File(retail));
num_rows = size(retail)[1] # 676
num_product = length(unique(retail[!,"product_id"])) # 52
demand = retail[!,"customers"]
cost = retail[!,"freight_price"]

C1 = retail[!,"comp_1"]
C2 = retail[!,"comp_2"]
C3 = retail[!,"comp_3"]

com1_cost = retail[!,"fp1"]
com2_cost = retail[!,"fp2"]
com3_cost = retail[!,"fp3"];

In [32]:
ori_price = retail[!,"unit_price"]
ori_price = reshape(ori_price, 12, 39)  # Reshape by column (12 columns, 39 rows)
ori_price = permutedims(ori_price);

In [7]:
demand = reshape(demand, 12, 39)  # Reshape by column (12 columns, 39 rows)
demand = permutedims(demand)  # Reshape by row

cost = reshape(cost, 12, 39)  # Reshape by column (12 columns, 39 rows)
cost = permutedims(cost)  # Reshape by row

C1 = reshape(C1, 12, 39)  # Reshape by column (12 columns, 39 rows)
C1 = permutedims(C1)  # Reshape by row

C2 = reshape(C2, 12, 39)  # Reshape by column (12 columns, 39 rows)
C2 = permutedims(C2)  # Reshape by row

C3 = reshape(C3, 12, 39)  # Reshape by column (12 columns, 39 rows)
C3 = permutedims(C3);  # Reshape by row

In [8]:
# Test on 1 product (12 months)

In [78]:
function geting_cost(C1,C2,C3)
        average_cost = (C1 .+ C2 .+ C3) ./ 3
    return average_cost
end

geting_cost (generic function with 1 method)

In [79]:
c= geting_cost(C1[1,:],C2[1,:],C3[1,:]);
l=cost[1, :]
d=demand[1, :];

In [80]:
# Constraint: x[t] - 1.05 * x[t-1] <= 0
function constraint_ineq(x)
    constraints = []
    for i in 2:length(x)
        push!(constraints, x[i] - 1.05 * x[i-1])
    end
    return constraints
end

constraint_ineq (generic function with 1 method)

In [81]:
# Constraint: x >= 0
function constraint_x_positive(x)
    return x
end

constraint_x_positive (generic function with 1 method)

In [129]:
# Gradient computation
function gradient_f(x, l, d, c)
    grad = similar(x)
    #print(grad)
    for i in eachindex(x)
        grad[i] = (d[i]*exp(-6x[i]/c[i])*(c[i]+6l[i]-6x[i]))/c[i]
    end
    #print("endgrad")
    return grad
end

function gradient_2_f(x, l, d, c)
    grad = similar(x)
    for i in eachindex(x)
        grad[i] = -(12*d[i]*exp(-6x[i]/c[i])*(c[i]+3l[i]-3x[i]))/c[i]^2
    end
    #print("endgrad")
    return grad
end

gradient_2_f (generic function with 1 method)

In [124]:
function gradient_g(x,mu)
    grad = similar(x)
    for i in eachindex(x)
        if i == 1
            grad[i] = -mu*(1/x[i])
        else
            grad[i] = -mu*(1/(1.05x[i-1]-x[i]))
        end
    end
    #print("endgrad")
    return grad
end

function gradient_2_g(x,mu)
    grad = similar(x)
    for i in eachindex(x)
        if i == 1
            grad[i] = u*(1/x[i]^2)
        else
            grad[i] = mu*(1/(1.05x[i-1]-x[i])^2)
        end
    end
    return grad
end

gradient_2_g (generic function with 1 method)

In [176]:
# Objective function: f(x) = (x - l) * d * (exp(-x / c))^2
function objective_function(x, l, d, c)
    result = 0
    for i in eachindex(x)
        result += ((x[i] - l[i]) * d[i] * (exp(-x[i] / c[i]))^2)
    end
    return result
end

objective_function (generic function with 1 method)

In [43]:
# Hessian computation
function hessian(grad_2_f,grad_2_g)
    #print(" hes ")
    n = length(grad_2_f)
    hess = zeros(n, n)
    for i in 1:n
        for j in 1:n
            if i==j
            hess[i,j] = grad_2_f[i]+grad_2_g[i]
            end
        end
    end
    #print(" hes_end ")
    return hess
end

hessian (generic function with 1 method)

In [44]:
# Hessian computation
function overall_gradient(grad_f,grad_g)
    grad = similar(grad_f)
    n = length(grad)
    for i in 1:n
        grad[i] = grad_f[i]+grad_g[i]
    end
    #print(" over-grad ")
    return grad
end

overall_gradient (generic function with 1 method)

In [214]:
# Interior point Newton's method
function interior_point_newton(x0, l, d, c, tol=1e-6, max_iter=200)
    x = copy(x0)  # Initial feasible solution
    mu = 1.0  # Penalty parameter
    
    for _ in 1:max_iter
        # Augmented Lagrangian function
        vector_f = gradient_f(x,l,d,c)
        #print("vector_f", vector_f)
        vector_b = gradient_g(x,mu)
        #print("vector_b", vector_b)
        vector_2_f = gradient_2_f(x,d,l,c)
        vector_2_b = gradient_2_g(x,mu)
        hessian_mtx=hessian(vector_2_f,vector_2_b)
        #print(vector_f)
        vector_total = overall_gradient(vector_f,vector_b)
        # Newton's method
        delta_x = -inv(hessian_mtx) * vector_total
        x_new = x + delta_x
        # Check convergence
        if norm(delta_x) < tol
            break
        end

        x = x_new
        mu *= .90  # Increase penalty for convergence
    end
    
    return x
end
# Initial guess for time series x
#initial_x = zeros(Float64, length(d))  # Replace this with your initial time series

interior_point_newton (generic function with 3 methods)

In [222]:
my_vector = fill(100.0, 12);

In [223]:
initial_x = ori_price[1,:]
# Run the interior point algorithm with Newton's method
optimized_x = interior_point_newton(my_vector,l,d,c)
println("optimial: ", optimized_x)

optimial: [28.958756485305372, 25.56876493850487, 25.044022713390458, 29.210651653204753, 34.59166485402468, 32.0916680465279, 33.696251611019115, 32.91860194433948, 31.72493092008346, 25.20000112645849, 25.330711842636312, 24.589352307471092]


In [206]:
print("intitial price" , initial_x)
print("\n")
print("optimal price: ", optimized_x)

intitial price[41.26681876660176, 41.61653094215304, 43.60909154298178, 45.75775106815832, 45.95, 45.95, 45.95, 45.95, 45.95, 45.95, 40.53181818, 39.99]
optimal price: [3.961408125713219e33, 2.0797392659994403e33, 1.0918631146497063e33, 5.732281351910959e32, 3.0094477097532534e32, 1.579960047620458e32, 8.294790250007407e31, 4.354764881253889e31, 2.286251562658291e31, 1.200282070395603e31, 6.301480869576917e30, 3.308277456527882e30]

In [201]:
print(objective_function(optimized_x,l,d,c))

6664.095364456399

In [185]:
function objective_function2(x, l, d, c)
    result = 0
    for i in eachindex(x)
        result += ((x[i] - l[i]) * d[i] * (exp(-x[i] / c[i]))^2)
        println(result)
    end
    return result
end

objective_function2 (generic function with 1 method)